In [1]:
import json
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pandas as pd
from selenium.webdriver.common.by import By

In [2]:
# wait
def wait(driver, selector_name):
    while (True):
        try:
            myElem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, selector_name)))
            if myElem is not None:
                break
        except TimeoutException:
            break

In [3]:
# scroll
def scroll(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [4]:
import sys

def printProgressBar(i,max,postText):
    n_bar =10 #size of progress bar
    j= i/max
    sys.stdout.write('\r')
    sys.stdout.write(f"[{'=' * int(n_bar * j):{n_bar}s}] {int(100 * j)}%  {postText}")
    sys.stdout.flush()

In [5]:
my_url = "https://soundcloud.com/discover"

In [7]:
driver.get(my_url)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
client_id = soup.find("div", {"id": "g_id_intermediate_iframe"}).get("data-src")
client_id = ((client_id.split('?')[1]).split('&')[0]).split('=')[1]

In [8]:
client_id

'vBo4hOqGwpN6821F16NG0ib45ueccYs2'

In [9]:
set_link = []

In [17]:
driver = webdriver.Chrome()
driver.get(my_url)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
link = soup.find_all("a", {
    "class": "sc-link-secondary sc-link-light playableTile__usernameHeading audibleTile__usernameHeading sc-truncate sc-text-h4"})

for item in link:
    user_url = "https://soundcloud.com" + item.get("href")
    if user_url not in set_link:
        set_link.append(user_url)

for user_url in set_link:
    driver.get(user_url)
    wait(driver, 'profileHeaderInfo__userName')
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    user_name = soup.find("h2", {"class": "profileHeaderInfo__userName"})

    # Get followers
    follower_url = user_url + "/following"
    driver.get(follower_url)

    #Wait
    scroll(driver)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    followers = soup.find_all("a", {"class": "userBadgeListItem__heading"})
    for follower in followers:
        if follower not in set_link:
            set_link.append("https://soundcloud.com" + follower.get("href"))
    if len(set_link) > 25:
        break

In [18]:
len(set_link)

67

In [19]:
def get_all_user_data(set_link):
    #Get user id in SoundCloud/Discover by get meta tag in HEAD html
    set_data_user = []
    
    for i, link_user in enumerate(set_link):
        driver = webdriver.Chrome()        
        driver.get(link_user)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        #Get user id & user name & user url
        id_user = soup.find("meta", property="twitter:app:url:googleplay")
        set_data_user.append(id_user["content"].strip("soundcloud://users:"))
        printProgressBar(i, len(set_link), "%")
        
    driver.close()
    
    return set_data_user

In [21]:
set_user = get_all_user_data(set_link)

KeyboardInterrupt: 

In [ ]:
len(set_user)

In [15]:
def get_user_by_API(set_user, client_id):
    set_data_user = []
    # Select first column of the dataframe as a series
    
    for i, item in enumerate(set_user):
        one_data_user = []
        user_id = item
        api_url = f'https://api-v2.soundcloud.com/users/{user_id}?client_id={client_id}'
        # get request
        r = requests.get(api_url)
        # check whether it get the url
        # or if it got request, does it have a string "incomplete_results":true in the response it recieved?
        while r.status_code != requests.codes.ok or '"incomplete_results":true' in r:
            time.sleep(1)
            r = requests.get(api_url)
        #JSON processing
        user = json.loads(r.text)
        #Get one data user
        one_data_user.append(user['id'])
        one_data_user.append(user['username'])
        one_data_user.append(user['permalink_url'])
        one_data_user.append(user['playlist_count'])
        one_data_user.append(user['track_count'])
        one_data_user.append(user['city'])
        one_data_user.append(user['followers_count'])
        one_data_user.append(user['followings_count'])
        one_data_user.append(user['likes_count'])
        one_data_user.append(user['avatar_url'])
        one_data_user.append(user['created_at'])
        one_data_user.append(user['kind'])
        
        set_data_user.append(one_data_user)
        printProgressBar(i, len(set_user), "%")
        
    return set_data_user

In [16]:
set_data_user = get_user_by_API(client_id)

In [17]:
f_out = pd.DataFrame(set_data_user, columns=['User ID',
                                             'User Name',
                                             'User URL',
                                             'playlist_count',
                                             'track_count',
                                             'CITY',
                                             'followers_count',
                                             'followings_count',
                                             'likes_count',
                                             'avatar_url',
                                             'created_at',
                                             'kind'
                                            ])
f_out.to_csv('user.csv', index=False, sep='\t', encoding='utf-8')

In [ ]:
get_user_by_API(set_user, client_id)

In [ ]:
def createNoneObj(n):
    return ['None' for i in range(n)]

In [ ]:
def getTracksAPI(set_user, client_id, limit=100):
    tracks = []
    temp1 = f'https://api-v2.soundcloud.com/users/'
    temp2 = f'/tracks?client_id={client_id}&limit={limit}'

    for user_id in set_user:
        api = temp1 + user_id + temp2
        while True:
            r = requests.get(api)
            if r.status_code != 200:
                time.sleep(1)
            else:
                break
        y = json.loads(r.text)
        if len(y['collection']) == 0:
            tracks.append([user_id] + createNoneObj(18))
        else:
            for track in y['collection']:
                tracks.append([
                    track['user_id'],
                    track['artwork_url'],
                    track['created_at'],
                    track['duration'],
                    track['genre'],
                    track['id'],
                    track['kind'],
                    track['last_modified'],
                    track['license'],
                    track['likes_count'],
                    track['permalink'],
                    track['permalink_url'],
                    track['public'],
                    track['reposts_count'],
                    track['sharing'],
                    track['tag_list'],
                    track['title'],
                    track['uri'],
                    track['display_date'],
                ])
    return tracks

In [ ]:
set_tracks = getTracksAPI(set_user, client_id)

In [ ]:
f_out = pd.DataFrame(set_data_user, columns=['user_id',
                                             'artwork_url',
                                             'created_at',
                                             'duration',
                                             'genre',
                                             'id',
                                             'kind',
                                             'last_modified',
                                             'license',
                                             'likes_count',
                                             'permalink',
                                             'permalink_url',
                                             'public',
                                             'reposts_count',
                                             'sharing',
                                             'tag_list',
                                             'title',
                                             'uri',
                                             'display_date'
                                            ])
f_out.to_csv('trackAPI.csv', index=False, sep='\t', encoding='utf-8')

In [16]:
def getPlaylistsAPI(set_user, client_id, limit=100):
    playlists = []
    
    for user_id in set_user:
        
        url = f'https://api-v2.soundcloud.com/users/{user_id}/playlists?client_id={client_id}&limit={limit}'
        r = requests.get(url)

        while r.status_code != requests.codes.ok or '"incomplete_results":true' in r:
            time.sleep(1)
            r = requests.get(url)

        y = json.loads(r.text)
        if len(y['collection']) == 0:
            playlists.append([user_id]+createNoneObj(24))
        else:
            for playlist in y['collection']:
                trackIDs = []
                for track in playlist['tracks']:
                    trackIDs.append(str(track['id']))
                allTrackIDs = ','.join(trackIDs)
                playlists.append([playlist['user_id'],
                                 playlist['artwork_url'],
                                 playlist['created_at'],
                                 playlist['duration'],
                                 playlist['embeddable_by'],
                                 playlist['genre'],
                                 playlist['id'],
                                 playlist['kind'],
                                 playlist['label_name'],
                                 playlist['last_modified'],
                                 playlist['license'],
                                 playlist['likes_count'],
                                 playlist['managed_by_feeds'],
                                 playlist['permalink'],
                                 playlist['permalink_url'],
                                 playlist['public'],
                                 playlist['reposts_count'],
                                 playlist['sharing'],
                                 playlist['tag_list'],
                                 playlist['title'],
                                 playlist['uri'],
                                 playlist['is_album'],
                                 playlist['published_at'],
                                 playlist['display_date'],
                                 allTrackIDs])

    return playlists

In [20]:
f_out = pd.DataFrame(temp, columns=['user_id',
                                 'artwork_url',
                                 'created_at',
                                 'duration',
                                 'embeddable_by',
                                 'genre',
                                 'id',
                                 'kind',
                                 'label_name',
                                 'last_modified',
                                 'license',
                                 'likes_count',
                                 'managed_by_feeds',
                                 'permalink',
                                 'permalink_url',
                                 'public',
                                 'reposts_count',
                                 'sharing',
                                 'tag_list',
                                 'title',
                                 'uri',
                                 'is_album',
                                 'published_at',
                                 'display_date',
                                 'allTrackIDs'])
f_out.to_csv('playlistApi.csv', index=False, sep='\t', encoding='utf-8')
